In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
data.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:

features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
x_data = pd.get_dummies(data[features]).astype(np.float32)
y_data = (data['Exited']).astype(np.float32)

scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)

x_train, x_, y_train, y_ = train_test_split(x_data, y_data, test_size=0.25)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

model_nn = Sequential(
    [
       
        Dense(256, activation = 'relu'),
         Dense(128, activation = 'relu'),
        Dropout(0.25),
        Dense(1, activation = 'sigmoid'),
    ]
)

model_nn.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
)

model_nn.fit(x_train, y_train, epochs=5)

Epoch 1/5
3868/3868 [==============================] - 14s 3ms/step - loss: 0.3692
Epoch 2/5
3868/3868 [==============================] - 11s 3ms/step - loss: 0.3349
Epoch 3/5
3868/3868 [==============================] - 11s 3ms/step - loss: 0.3307
Epoch 4/5
3868/3868 [==============================] - 11s 3ms/step - loss: 0.3290
Epoch 5/5
3868/3868 [==============================] - 11s 3ms/step - loss: 0.3267


In [5]:
test_data = pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
x_test = pd.get_dummies(test_data[features]).astype('float32')

x_test = scaler.transform(x_test)

In [6]:
test_pred = model_nn.predict(x_test)

3439/3439 [==============================] - 5s 1ms/step


In [7]:
[test_pred] = test_pred.reshape(1, len(test_pred))
test_pred = np.round(test_pred, 1)

output = pd.DataFrame({'id': test_data.id, 'Exited': test_pred})
output.to_csv('submission.csv', index=False)